In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import math
import numpy as np
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import sys
import os
import mlflow
from mlflow import tracking
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow.sklearn


In [2]:
url1="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"


In [3]:
tracking.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("pipeline homework experiment")

<Experiment: artifact_location='/workspaces/mlops/homework-pipelines/mlruns/1', creation_time=1749194083092, experiment_id='1', last_update_time=1749194083092, lifecycle_stage='active', name='pipeline homework experiment', tags={}>

In [4]:
df1=pd.read_parquet(url1)
print (f"yellow taxie has {len(df1)} records")

yellow taxie has 3403766 records


In [5]:
df1.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.0,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.4,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.3,1.0,N,140,186,1,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00


In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df=read_dataframe(url1)

In [8]:
print (f"yellow taxie has {len(df)} records")

yellow taxie has 3316216 records


In [9]:
df.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.0,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.4,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333


In [10]:
df.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
duration                        float64
dtype: object

In [11]:
categorical=["PULocationID","DOLocationID"]

train_dicts=df[categorical].to_dict(orient="records")
dv=DictVectorizer()
X_train=dv.fit_transform(train_dicts)


In [12]:
target="duration"
y_train=df[target].values



Training a model

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
import mlflow.sklearn


with mlflow.start_run():
    mlflow.set_tag("dev", "Ethel")
    mlflow.set_tag("model", "Linearregression")
    model=LinearRegression()
    model.fit(X_train, y_train)  
    inter=model.intercept_
    mlflow.sklearn.log_model(model, "linear")
    print(f"the intercept of the model is {inter}")

2025/06/06 08:13:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


the intercept of the model is 24.776368754137366
